# 中文情感分析
### 步骤
    1、读取评论数据集
    2、数据预处理(去空、去重)
    3、机械压缩
    4、短句过滤
    5、snownlp库——打分
    6、打分后分词
    7、gensim库——分析

### 知识点
    1、pandas读取数据
    2、空值删除
    3、去重复值（两个）
    4、apply函数、lambda函数
    5、机械压缩去词
    6、短句过滤
    7、snownlp库——进行情感分析【打分】
    8、打分后分词
    9、读取停用词文件
    10、去停用词
    11、gensim库——进行情感分析
### 不懂的地方
    1、两个函数(apply函数、lambda函数)
    2、机械压缩的含义/解释
    3、情感分析的两个库【打分库，分析库】
    4、分析后的数值
    5、分词后的数据为什么被分成几份了

### 笔记

[删除某列中含有空值的行][1]

[1]:https://blog.csdn.net/weixin_43474731/article/details/100155593

In [3]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_csv(r'F:\陶志远\数据\数据集\B站流行动漫影评数据\B站流行动漫影评数据.csv')
df

,author,score,disliked,likes,liked,ctime,content,last_ep_index,cursor,date
0,緖山まひろ,10,0,9981,0,1530982477,我有个想法，动物体内的细胞会不会是兽耳呢d(ŐдŐ๑),0,77395314810369,2018-07-07 16:54:37
1,onlylove丶亚丝娜,10,0,1,0,1557139155,题材新颖,0,77395314810369,2019-05-06 10:39:15
2,AKAHANE_铃木羽,10,0,3,0,1556947242,坐等第二季！！表白白细胞∠( ᐛ 」∠)＿,0,77395314810369,2019-05-04 05:20:42
3,妖怪退散,10,0,0,0,1557182714,好看，推荐,0,77395314810369,2019-05-06 22:45:14
4,受虎大王,10,0,0,0,1557118709,涨知识,0,77395314810369,2019-05-06 04:58:29
...,...,...,...,...,...,...,...,...,...,...
19058,八奇大蛇,2,0,1,0,1533969368,不配,0,74186973181452,2018-08-11 06:36:08
19059,我不听我就不改名,2,0,0,0,1533473635,第一次对一个网站以及一部番恶心到这种地步,0,74186973181452,2018-08-05 12:53:55
19060,截教至圣通天教主,2,0,0,0,1530984474,还可以,0,71618582349408,2018-07-07 17:27:54
19061,小寒、很忙,8,0,0,0,1530983251,血小板太萌了,0,71618582349408,2018-07-07 17:07:31


In [11]:
'''删除不需要的数据'''

del_col = ['disliked','likes','liked','ctime','last_ep_index','cursor','date']   # 要删除的列
df = df.drop(del_col, axis=1)

In [12]:
df2 = df.dropna()
df2

,author,score,content
0,緖山まひろ,10,我有个想法，动物体内的细胞会不会是兽耳呢d(ŐдŐ๑)
1,onlylove丶亚丝娜,10,题材新颖
2,AKAHANE_铃木羽,10,坐等第二季！！表白白细胞∠( ᐛ 」∠)＿
3,妖怪退散,10,好看，推荐
4,受虎大王,10,涨知识
...,...,...,...
19058,八奇大蛇,2,不配
19059,我不听我就不改名,2,第一次对一个网站以及一部番恶心到这种地步
19060,截教至圣通天教主,2,还可以
19061,小寒、很忙,8,血小板太萌了


In [15]:
# 去掉第一列的重复数据；iloc[:, 0]表示索引每一行的第一列；
df3 = pd.DataFrame(df2.iloc[:, 1].unique()) 
print('去重数据后：')
print(len(df))
df3

去重数据后：
19063


,0
0,10
1,8
2,6
3,2
4,4


In [16]:
#首先定义一个函数，去除重复数据。
def str_unique(raw_str, reverse=False):
    """
    比如：我喜欢喜欢喜欢喜欢喜欢喜欢该商品；去掉重复的“喜欢”
    :param raw_str:
    :param reverse: 是否转置
    :return:
    """
    if reverse:
        raw_str = raw_str[::-1]
    res_str = ''
    for i in raw_str:
        if i not in res_str:
            res_str += i
    if reverse:
        res_str = res_str[::-1]
    return res_str

In [17]:
#使用 apply 方法应用函数；
ser1 = df3.iloc[:, 0].apply(str_unique) # 这时，因为索引了第一列，所以结果成了 Series；
print('df2', type(ser1)) # <class 'pandas.core.series.Series'>
df4 = pd.DataFrame(ser1.apply(str_unique, reverse=True)) # 再次生成 DataFrame；
print('机械压缩去词后：')
print(len(df4))
print(type(df4))
print('------------------')
df4

TypeError: 'int' object is not iterable

In [ ]:
df5 = df4[df4.iloc[:, 0].apply(len) >= 4]
print('短句过滤后：')
print(len(df5))

In [ ]:
from snownlp import SnowNLP
coms = df5.iloc[:, 0].apply(lambda x: SnowNLP(x).sentiments)
print('情感分析后')
positive_df = df5[coms >= 0.9]
negative_df = df5[coms <= 0.1]
print('特别喜欢的')
print(positive_df)
print('___________________________')
print('不喜欢的')
print(negative_df)

In [ ]:
import jieba
my_cut = lambda s:' '.join(jieba.cut(s))
positive_ser = positive_df.iloc[:, 0].apply(my_cut)
negative_ser = negative_df.iloc[:, 0].apply(my_cut)
print('大于0.9——正面数据——分词')
print(positive_ser)
print('小于0.1——负面数据——分词')
print(negative_ser)

In [ ]:
stops = pd.read_csv(r'E:\Jupyter Notebook\常用表格文件\漂流瓶\stoplist2.txt', encoding='UTF-8', header=None, sep='tipdm', engine='python')
stops

In [ ]:
stops = [' ', ' '] +list(stops[0])
positive_df = pd.DataFrame(positive_ser)
negative_df = pd.DataFrame(negative_ser)
positive_df[1] = positive_df[0].apply(lambda s: s.split(' '))
positive_df[2] = positive_df[1].apply(lambda x: [i for i in x if i.encode('UTF-8') not in stops])

# 这里的```lambda```函数实在不懂

In [ ]:
print('=============================================================')
print('去停用词后：positive_df')
print(positive_df)
print('=============================================================')
print('去停用词后：negative_df')
print(negative_df)
print('=============================================================')

### gensim库——进行主题分析，分析出人们对商品及其属性的情感倾向。 

In [ ]:
from gensim import corpora, models

# 正面主题分析
pos_dict = corpora.Dictionary(positive_df[2])    # 建立词典
pos_corpus = [pos_dict.doc2bow(i) for i in positive_df[2]]    # 建立语料库
pos_lda = models.LdaModel(pos_corpus, num_topics=3, id2word = pos_dict)    #LDA 模型训练
print('正面主题分析')
for i in range(3):
    print('topic', i)
    print(pos_lda.print_topic(i))    # 输出每个主题

# 负面主题分析
neg_dict = corpora.Dictionary(negative_df[2])  # 建立词典
neg_corpus = [neg_dict.doc2bow(i) for i in negative_df[2]]    # 建立语料库
neg_lda = models.LdaModel(neg_corpus, num_topics=3, id2word = neg_dict)    #LDA 模型训练
print('负面主题分析')
for i in range(3):
    print('topic', i)
    print(neg_lda.print_topic(i))    # 输出每个主题

In [ ]:
import numpy as np
import pandas as pd

#### 读取数据

In [ ]:
import pandas as pd
df = pd.read_csv(r'F:\陶志远\同步文件夹\数据集\漂流瓶\comments.csv')

#### 缺失值删除
[使用pandas中的 dropna 方法过滤缺失数据][1]

[Pandas详解十之Dropna滤除缺失数据][2]

[1]:https://blog.csdn.net/weixin_44706915/article/details/102688651?utm_medium=distribute.pc_relevant.none-task-blog-BlogCommendFromMachineLearnPai2-3.nonecase&depth_1-utm_source=distribute.pc_relevant.none-task-blog-BlogCommendFromMachineLearnPai2-3.nonecase
[2]:https://blog.csdn.net/weixin_38168620/article/details/79596798

###### dropna() 函数
```df.dropna()```

默认删除行

参数:axis \ how \ thresh \ subset \ inplace

    axis = 0  表示删除行
    axis = 1  表示删除列
    
    how = 'any'  表示存在NAN就删除
    how = 'all'  表示全部为NAN才删除

In [ ]:
#生成一个4行5列的DataFrame数据，注意列名的写法
df = pd.DataFrame(np.random.randn(4,5), columns=list('abcde'))
df.loc[1,['b', 'd']] = np.nan    #生成NAN
df

In [ ]:
#默认删除所有含有NAN的行
df1 = df.dropna()
df1

In [ ]:
#删除所有含有NAN的行
#axis = 0
df2 = df.dropna(axis = 0) 
df2

In [ ]:
#删除所有含有NAN的列
#axis = 1
df3 = df.dropna(axis=1)
df3

In [ ]:
#增加一列全部为NAN
df4 = df
df4['f'] = np.nan
df4

In [ ]:
# how='any'
# 只要含有NAN就删除
df5 = df4.dropna(axis=0, how='any') # 搜索含有NAN的行，然后删除行
df5

In [ ]:
# how='any'
# 只要含有NAN就删除
df6 = df4.dropna(axis=1, how='any') # 搜索含有NAN的列，然后删除列
df6

In [ ]:
# how='all'
# 每行都含NAN的列
df7 = df4.dropna(axis=1, how='all')
df7

### 步骤
    1、读取评论数据集
    2、数据预处理(去空、去重)
    3、机械压缩
    4、短句过滤
    5、snownlp库——打分
    6、打分后分词
    7、gensim库——分析

### 知识点
    1、pandas读取数据
    2、空值删除
    3、去重复值（两个）
    4、apply函数、lambda函数
    5、机械压缩去词
    6、短句过滤
    7、snownlp库——进行情感分析【打分】
    8、打分后分词
    9、读取停用词文件
    10、去停用词
    11、gensim库——进行情感分析
### 不懂的地方
    1、两个函数(apply函数、lambda函数)
    2、机械压缩的含义/解释
    3、情感分析的两个库【打分库，分析库】
    4、分析后的数值
    5、分词后的数据为什么被分成几份了

### 去重复值

In [ ]:
#生成一个4行5列的DataFrame数据，注意列名的写法
af = pd.DataFrame([[2,3,'你'],[2,4,6],['a','t','y'],['你','在','你']], columns=list('beg'))
af

In [ ]:
af2 = af.iloc[:, 2].unique()
pd.DataFrame(af2)